![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FMLOps&file=Vertex+AI+Pipelines+-+Introduction.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/MLOps/Vertex%20AI%20Pipelines%20-%20Introduction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FMLOps%2FVertex%2520AI%2520Pipelines%2520-%2520Introduction.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/MLOps/Vertex%20AI%20Pipelines%20-%20Introduction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/MLOps/Vertex%20AI%20Pipelines%20-%20Introduction.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# Vertex AI Piplines - Introduction

When an ML workflow has more than one step it can benefit from a pipeline.  [Vertex AI Pipelines]() is a managed service that executes [Kubeflow Pipelines (KFP)](https://www.kubeflow.org/docs/components/pipelines/v2/introduction/) and [TensorFlow Extended (TFX)](https://www.tensorflow.org/tfx/guide/understanding_tfx_pipelines) pipelines.  For this introduction the focus will be on KFP due to its vast flexibility, it was even originally developed as a simplified way of running TFX on Kubernetes - [history of Kubeflow](https://www.kubeflow.org/docs/started/introduction/#history).

This notebook based workflow will introduce KFP pipelines runninng on Vertex AI Pipelines and point to the additional detailed workflows within [this repository](./readme.md) that help with a deeper understanding.

---

**Pipelines** - The Concept

Pipelines are constructed from steps.  The steps are called **tasks** and utilize **components**.

The tasks are connected by inputs and outputs.  This forms the execution graph of the pipeline.

Inputs and outputs can be made up of **parameters** (`str`, `int`, `float`, `bool`, `dict`, `list`) as well as **artifacts**.  Artifacts are multiparameter objects that have defined schemas for machine learning objects (datasets, models, etc.) and automatically get stored as [Vertex AI ML Metadata](https://cloud.google.com/vertex-ai/docs/ml-metadata/introduction) and have lineage.

**Kubeflow Pipelines** - Constructing Pipelines

Pipelines are written in Python using the [`kfp` package](https://www.kubeflow.org/docs/components/pipelines/v2/installation/).  This SDK has decorators for multiple ways of writing [**components**](https://www.kubeflow.org/docs/components/pipelines/v2/components/) and constructing [**pipelines**](https://www.kubeflow.org/docs/components/pipelines/v2/pipelines/) from the components with features to control the execution and flow of the pipeline. Pipeline code is [**compiled**](https://www.kubeflow.org/docs/components/pipelines/v2/compile-a-pipeline/) into a YAML file using a single line command with the SDK.  

**Vertex AI Pipelines** - Executing Pipelines

The resulting YAML file is the input for [running the pipeline](https://cloud.google.com/vertex-ai/docs/pipelines/run-pipeline) with Vertex AI Pipelines.  The runs can even be scheduled with [Vertex AI Pipeline scheduler API](https://cloud.google.com/vertex-ai/docs/pipelines/schedule-pipeline-run).

The runs of a pipeline can be directly monitoring in the Vertex AI console!

screen capture of pipeline console here

---
## Colab Setup

To run this notebook in Colab run the cells in this section.  Otherwise, skip this section.

This cell will authenticate to GCP (follow prompts in the popup).

In [1]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [2]:
try:
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
    print('Colab authorized to GCP')
except Exception:
    print('Not a Colab Environment')
    pass

Not a Colab Environment


---
## Installs

The list `packages` contains tuples of package import names and install names.  If the import name is not found then the install name is used to install quitely for the current user.

In [3]:
# tuples of (import name, install name, min_version)
packages = [
    ('google.cloud.aiplatform', 'google-cloud-aiplatform'),
    ('kfp', 'kfp'),
]

import importlib
install = False
for package in packages:
    if not importlib.util.find_spec(package[0]):
        print(f'installing package {package[1]}')
        install = True
        !pip install {package[1]} -U -q --user
    elif len(package) == 3:
        if importlib.metadata.version(package[0]) < package[2]:
            print(f'updating package {package[1]}')
            install = True
            !pip install {package[1]} -U -q --user

## API Enablement

In [4]:
!gcloud services enable aiplatform.googleapis.com

### Restart Kernel (If Installs Occured)

After a kernel restart the code submission can start with the next cell after this one.

In [5]:
if install:
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

---
## Setup

Inputs

In [6]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [7]:
REGION = 'us-central1'
EXPERIMENT = 'pipeline-intro'
SERIES = 'mlops'

# gcs bucket
GCS_BUCKET = PROJECT_ID

Packages

In [8]:
import os
import time
import importlib
from google.cloud import aiplatform
import kfp
from typing import NamedTuple

In [9]:
kfp.__version__

'2.7.0'

In [10]:
aiplatform.__version__

'1.50.0'

Clients

In [11]:
# vertex ai clients
aiplatform.init(project = PROJECT_ID, location = REGION)

parameters:

In [12]:
DIR = f"temp/{SERIES}-{EXPERIMENT}"

In [13]:
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

'1026793852137-compute@developer.gserviceaccount.com'

environment:

In [14]:
if not os.path.exists(DIR):
    os.makedirs(DIR)

---
## KFP Pipeline On Vertex AI Pipelines

An example workflows: a KFP pipeline constructed from custom components and run on Vertex AI Pipelines.